<a href="https://colab.research.google.com/github/angelopasc/Progetti-AP/blob/main/Progetto_Analisi_incidenti_aerei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L'obiettivo di questo lavoro è analizzare il dataset contenente i disastri aerei avvenuti dal 1919 al 2023. Il dataset è in formato csv ed è stato precedentemente caricato su github, per potervi accedere tramite url.

Importiamo le varie librerie che ci serviranno per la fase di importazione e lettura del dataset e passiamo alla seconda fase, di pulizia e manipolazione dei dati.


In [ ]:
pip install thefuzz

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from thefuzz import process
import geopandas as gpd

csv_url = 'https://raw.githubusercontent.com/angelopasc/ProAI-Progetto-2/main/aviation-accidents.csv'
df = pd.read_csv(csv_url)
del df['registration']

df

Una volta importato e visualizzato il dataset, veniamo a conoscenza della sua dimensione: 8 colonne e 23967 righe. Le colonne contengono le informazioni di data, tipo del veicolo, operatore, numero di morti, località, nazione, categoria e anno. Iniziamo la procedura di pulizia andando a sostituire il valore Nan sulle righe che contengono valori di data e anno nulli e a cancellarle.

In [ ]:
#rimozione delle colonne con data "date unk."
df.loc[(df['date'].str.contains('\?', na=False) | (df['date'] == 'date unk.')), 'date'] = pd.NaT
#pulizia delle righe vuote

df = df.dropna()
df.info()


Dalle info del dataset notiamo che è necessario modificare i formati di due colonne: la colonna 'date' va modificata in formato datetime e la colonna 'fatalities' in int. Quando andiamo a richiedere la modifica della colonna 'fatalities' in int, notiamo che alcune colonne sono scritte come stringhe contenenti un'addizione, ad esempio "5 + 1". Scriviamo una funzione a cui passiamo il contenuto della colonna fatalities e per ogni valore controlla la presenza eventuale di un segno '+'. Se presente divide la stringa per valore '+', elimina gli spazi bianchi con il metodo strip e effettua la somma delle parti presenti nella lista creata con lo split.

In [ ]:
#conversione delle colonne nel formato corretto

def convert_to_int(value):
    if '+' in value:
        # Divide la stringa in parti, rimuove eventuali spazi bianchi e converte ogni parte in intero
        parts = value.split('+')
        return sum(int(x.strip()) for x in parts)
    else:
        # Se il valore non contiene un segno di più, lo converte direttamente in intero
        return int(value)


df['date'] = pd.to_datetime(df['date'], format= '%d-%b-%Y')
df["fatalities"]=df['fatalities'].apply(convert_to_int)

df.info()

Adesso possiamo procedere con la creazione di alcuni elementi di data visualization. Partiamo dalla creazione di un barchart mettendo in relazione anno con il numero di incidenti nell'anno.


In [ ]:
plt.figure(figsize=(20, 7))

df['year'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('Anno')
plt.title('Numero di incidenti per anno')
plt.ylabel('Numero di Incidenti')
plt.xticks(rotation=90)
plt.text(70,600, "Il 1944 rappresenta l'anno con più incidenti nella storia", size=10, bbox=dict(boxstyle="square, pad=0.5", facecolor='lightgrey'))
plt.show()

Creiamo uno scatterplot che mette in relazione il numero di morti per anno. Notiamo che il 2001 è l'anno con frequenza maggiore. Questo può farci pensare che nel conteggio abbia inciso il disastro dell'11 settembre 2001. Andiamo ad esplorare ulteriormente i dati per verificare questa ipotesi.

In [ ]:
plt.figure(figsize=(20, 7))
plt.plot(df['year'].unique(), df.groupby('year')['fatalities'].sum())
plt.scatter(df['year'].unique(), df.groupby('year')['fatalities'].sum())
plt.text(90,3800, "Il massimo è ottenuto nel 2001", size=10, bbox=dict(boxstyle="square, pad=0.5", facecolor='lightgrey'))
plt.title('Numero di morti per anno')
plt.xlabel('Anno')
plt.ylabel('Numero di morti')
plt.xticks(rotation=90)
plt.grid()
plt.show()

Di fatti, cercando il volo che contenga la frequenza di morti più alta, notiamo che il valore che ci ritorna è proprio quello. Dal grafico precedente notiamo che successivamente a quel picco, il numero di morti è andato via via diminuendo, benchè il numero di incidenti sia rimasto più o meno stabile, ad eccezione del valore basso del 2023.

In [ ]:
max_fatalities_index = df['fatalities'].idxmax()
max_fatalities_id = df.loc[max_fatalities_index]
max_fatalities_id

Inseriamo un grafico che metta in relazione il paese con il numero di incidenti. Essendo tantissime , mostriamo solo le nazioni con frequenza superiore alla media.

In [ ]:
plt.figure(figsize=(20, 7))
df_by_country = df['country'].value_counts()
country_over_mean = df_by_country[df_by_country > df_by_country.mean()]
plt.title('Numero di incidenti per nazione')

country_over_mean.plot(kind='bar')
plt.xlabel('Nazione')
plt.ylabel('Numero di Incidenti')
plt.xticks(rotation=90)
plt.text(28,3800, "La nazione in cui si sono verificati più incidenti è gli USA, seguiti da Russia e Canada", size=10, bbox=dict(boxstyle="square, pad=0.5", facecolor='lightgrey'))

plt.show()


Facciamo la stessa cosa per analizzare i modelli più soggetti ad incidente. Inseriamo un grafico che metta in relazione il modello con il numero di incidenti. Essendo tantissime modalità, mostriamo solo i modelli con più di 100 incidenti.

In [ ]:
df['type'].value_counts().mean()


plt.figure(figsize=(20, 7))
df_by_type= df['type'].value_counts()
type_over_mean = df_by_type[df_by_type > 100]

plt.title('Numero di incidenti per tipo di veicolo')
type_over_mean.plot(kind='bar')
plt.xlabel('Tipo di veivolo')
plt.ylabel('Numero di Incidenti')
plt.xticks(rotation=90)
plt.text(10,1000, "Gli incidenti aerei si sono verificati maggiormente nei veivoli Douglas, che occupano i primi 3 posti", size=10, bbox=dict(boxstyle="square, pad=0.5", facecolor='lightgrey'))

plt.show()

Dalla colonna datetime estraiamo il giorno della settimana e lo mettiamo in relazione al numero di incidenti su un line chart. Abbiamo un andamento tutto sommato costante con un calo la domenica, probabilmente relativo al fatto che di domenica potrebbero partire meno voli.

In [ ]:
df.loc[:, 'day_of_week'] = df['date'].dt.strftime('%A')
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
x = df['day_of_week'].value_counts().reindex(days_order, fill_value=0)
plt.title('Numero di incidenti per giorno della settimana')
plt.ylim(1000,4000)
plt.plot(x)
plt.show()

Su di un pie chart mettiamo in relazione la percentuale degli incidenti mortali con la percentuale degli incidenti non mortali, con una leggera prevalenza degli incidenti non mortali.

In [ ]:

plt.figure(figsize=(20, 7))

x = df.loc[df['fatalities'] == 0,'fatalities'].shape[0]
y = df.loc[df['fatalities'] != 0,'fatalities'].shape[0]
labels = ["Non mortali","Mortali"]
plt.title("Rapporto tra Incidenti mortali e non mortali")
plt.text(-1.3,-1.2, "Gli incidenti aerei in una leggera maggioranza dei casi non sono stati mortali", size=10, bbox=dict(boxstyle="square, pad=0.5", facecolor='lightgrey'))

print(x)
print(y)
plt.pie([x,y], labels = labels, autopct='%1.1f%%', startangle=90)
plt.show()

Per creare una mappa che mostri il numero di incidenti per nazione, è stato utilizzato geopandas. Importando il dataset contenente i nomi delle nazioni e le forme degli stati, è stato effettuato un merge tra il dataset delle nazioni importato e un dataframe contenente nazione e numero di incidenti. A questo punto è venuto alla luce un problema di formattazione tra le nazioni scritte nel dataframe e le stringhe delle nazioni contenute nel file geografico. E' stata utilizzata la libreria thefuzz per utilizzare una logica fuzzy all'80% di somiglianza per il riconoscimento di gran parte degli stati. Alcuni sono stati integrati a mano, altri (minori) non sono stati toccati, scegliendo di trascurarli. Il risultato ottenuto è coerente con quello presente nel grafico a barre precedente, con una netta prevalenza degli incidenti negli USA, seguiti da Russia e Canada.

In [ ]:


world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Nomi unici dei paesi del DF
unique_countries_df = world['name'].unique()

# Funzione per impostare la logica fuzzy all'80%
def get_best_match(query, choices):
    match, score = process.extractOne(query, choices)
    return match if score > 80 else query


country_mapping = {country: get_best_match(country, unique_countries_df) for country in unique_countries_df}

df = df.copy()
df['country'] = df['country'].replace(country_mapping)

#Sostituzione manuale di alcune label sfuggite
country_mapping = {
    'USA': 'United States of America',
    'Italia': 'Italy',
    'U.K.': 'United Kingdom',
}

df['country'] = df['country'].replace(country_mapping)

#Visualizzazione dei paesi non corrispondenti
remaining_discrepancies = set(df['country']) - set(unique_countries_df)

print(f'\n Paesi ancora non corrispondenti:' ,remaining_discrepancies)



In [ ]:

# Conta il numero di incidenti per paese
df_by_country = df['country'].value_counts()

# Converti il conteggio in un DataFrame
df_by_country = df_by_country.reset_index()
df_by_country.columns = ['country', 'incidents']

# Carica il file shape del mondo
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Unisci il DataFrame con il GeoDataFrame
merged = world.set_index('name').join(df_by_country.set_index('country'))

# Crea la mappa
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=1)
merged.plot(column='incidents', ax=ax, legend=True, legend_kwds={'label': "Numero di incidenti", 'orientation': "horizontal"}, cmap='YlOrRd', missing_kwds={"color": "lightgrey"})
ax.axis('off')

plt.title('Numero di incidenti per paese')
plt.show()